In [9]:
#!pip install oauth2client

In [10]:
#pip install google-cloud-storage

In [12]:
#!pip install twilio

In [1]:
#!pip install google-api-python-client

In [2]:
# import os
# from twilio.rest import Client


# # Find your Account SID and Auth Token at twilio.com/console
# # and set the environment variables. See http://twil.io/secure


# account_sid = 'AC67576f379fa5f28163b8f79286326d60'
# auth_token = 'b33aedc711d08e0ed4fe798e504cd99e'
# client = Client(account_sid, auth_token)

# message = client.messages \
#                 .create(
#                      body="Join Earth's mightiest heroes. Like Kevin Bacon.",
#                      from_='+16692807604',
#                      to='+917001427461'
#                  )

# print(message.sid)

In [1]:
import os
PATH=os.path.join(os.getcwd(),'master-shell-352711-035a6f0dc8c6.json')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=PATH

In [3]:
# folder_list=['suman','amon']         
# created_folder=','.join(folder_list)
# created_folder=created_folder+'- folder has been created'    
# updated_msg=created_folder

# account_sid = 'AC67576f379fa5f28163b8f79286326d60'
# auth_token = 'b33aedc711d08e0ed4fe798e504cd99e'
# client = Client(account_sid, auth_token)

# message = client.messages \
#                 .create(
#                      body=updated_msg,
#                      from_='+16692807604',
#                      to='+917001427461'
#                  )

# print(message.sid)

In [4]:
from google.cloud import storage
from google.cloud.storage import Blob

import io
import os 
from google.cloud.storage import Blob
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload

In [6]:
# def create_bucket(bucket_name):
#     storage_client = storage.Client()
#     new_bucket = storage_client.create_bucket(bucket_name)

#     print(
#       "Created bucket successfully {}".format(
#           new_bucket.name
#       )
#     )
    
    
# buck_name=['amitbiswas1933']
# for item in buck_name:
#     create_bucket(item)

In [18]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']



def get_blobs():
    client = storage.Client() 
    bucket_name = client.get_bucket("practise_bucket3456")
    objects_in_bucket = client.list_blobs(bucket_name, fields='items(name)')
    object_names = []
    for object in objects_in_bucket:
        object_names.append(object.name)
    return object_names




def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        drive = build('drive', 'v3', credentials=creds)
        
        files = []
        page_token = None
        file_to_download = None
        objects_list = get_blobs()
        
        while True:
    
            folderId = drive.files().list(q = "mimeType = 'application/vnd.google-apps.folder' and name = 'test_pipeline'", pageSize=10, fields="nextPageToken, files(id, name)").execute()
            folderIdResult = folderId.get('files', [])

            id = folderIdResult[0].get('id')
            results = drive.files().list(q = "'" + id + "' in parents", pageSize=10, fields="nextPageToken, files(id, name)").execute()

            items = results.get('files', [])  

            if not items:
                print('No files found.')
                return
            print('Files:')
            for item in items:
                print(u'{0} ({1})'.format(item['name'], item['id']))
                file_id = item.get("id")
                file_name = item.get("name")
                if file_name not in objects_list:
                    request = drive.files().get_media(fileId=file_id)
                    file_download_buffer = io.BytesIO()
                    downloader = MediaIoBaseDownload(file_download_buffer, request)
                    done = False
                    while done is False:
                        status, done = downloader.next_chunk()
                        print(F'Download {int(status.progress() * 100)}.')

                    upload_to_drive(file_download_buffer,file_name)

            files.extend(folderId.get('files', []))
            page_token = folderId.get('nextPageToken', None)


            if page_token is None:
                break
         
            
            
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')

        
        
# def upload_to_drive(file, file_name):
#     client = storage.Client()
#     bucket_name = client.get_bucket("practise_bucket3456")
#     blob = Blob(file_name, bucket_name)
#     blob.upload_from_file(file, rewind=True)
    
    
    
    
def upload_to_drive(file, file_name):
    client = storage.Client()
    bucket_name = client.get_bucket("practise_bucket3456")
#     blob = Blob(file_name, bucket_name)
    blob = bucket_name.blob("amit/"+file_name)
    #blob.upload_from_filename(source_file_name)
    blob.upload_from_filename(file, rewind=True)
    
    
    

if __name__ == '__main__':
    main()

Files:
car_company.csv (1VxfujWkjWpQZ-Tm7OeMqA45UTsuzAMU0)
